##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Classificação de flores com aprendizado por transferência


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/image_feature_vector"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

Alguma vez você já viu uma linda flor e quis saber que tipo era? Bem, você não é a primeira pessoa a indagar, então criamos uma forma de identificar o tipo de flor a partir de uma foto!

Para classificação de imagens, um tipo específico de *rede neural profunda*, chamado de *rede neural convolucional*, provou ser especialmente poderoso. Entretanto, as redes neurais convolucionais modernas têm milhões de parâmetros. Treiná-las do zero requer muitos dados de treinamento rotulados e muito poder computacional (centenas de horas de GPU ou mais). Temos somente cerca de 3 mil fotos rotuladas e queremos gastar muito menos tempo, então precisamos ser mais inteligentes.

Usaremos uma técnica chamada *aprendizado por transferência*, em que pegamos uma rede pré-treinada (usando cerca de um milhão de imagens gerais), a usamos para extrair características e treinamos uma nova camada para nossa própria tarefa de classificação de imagens de flores.


## Configuração


In [ ]:
import collections
import io
import math
import os
import random
from six.moves import urllib

from IPython.display import clear_output, Image, display, HTML

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import time

## Dataset Flowers

O dataset Flowers consiste de imagens de flores com 5 rótulos de classes possíveis.

Ao treinar um modelo de aprendizado de máquina, dividimos os dados em datasets de treinamento e teste. Vamos treinar o modelo com nossos dados de treinamento e depois avaliar o desempenho do modelo com dados jamais observados – o conjunto de teste.

Vamos baixar os exemplos de treinamento e teste (pode demorar um pouco) e dividi-los em datasets de treinamento e teste.

Execute as duas células abaixo:

In [ ]:
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018


def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)


def make_train_and_test_sets():
  """Split the data into train and test sets and get the label classes."""
  train_examples, test_examples = [], []
  shuffler = random.Random(RANDOM_SEED)
  is_root = True
  for (dirname, subdirs, filenames) in tf.gfile.Walk(FLOWERS_DIR):
    # The root directory gives us the classes
    if is_root:
      subdirs = sorted(subdirs)
      classes = collections.OrderedDict(enumerate(subdirs))
      label_to_class = dict([(x, i) for i, x in enumerate(subdirs)])
      is_root = False
    # The sub directories give us the image files for training.
    else:
      filenames.sort()
      shuffler.shuffle(filenames)
      full_filenames = [os.path.join(dirname, f) for f in filenames]
      label = dirname.split('/')[-1]
      label_class = label_to_class[label]
      # An example is the image file and it's label class.
      examples = list(zip(full_filenames, [label_class] * len(filenames)))
      num_train = int(len(filenames) * TRAIN_FRACTION)
      train_examples.extend(examples[:num_train])
      test_examples.extend(examples[num_train:])

  shuffler.shuffle(train_examples)
  shuffler.shuffle(test_examples)
  return train_examples, test_examples, classes


In [ ]:
# Download the images and split the images into train and test sets.
download_images()
TRAIN_EXAMPLES, TEST_EXAMPLES, CLASSES = make_train_and_test_sets()
NUM_CLASSES = len(CLASSES)

print('\nThe dataset has %d label classes: %s' % (NUM_CLASSES, CLASSES.values()))
print('There are %d training images' % len(TRAIN_EXAMPLES))
print('there are %d test images' % len(TEST_EXAMPLES))

## Explore os dados

O dataset Flowers consiste de exemplos que são imagens de flores rotuladas. Cada exemplo contém uma imagem de flor em JPEG e o rótulo de classe: o tipo de flor. Vamos exibir algumas imagens juntamente com os respectivos rótulos.

In [ ]:
#@title Show some labeled images
def get_label(example):
  """Get the label (number) for given example."""
  return example[1]

def get_class(example):
  """Get the class (string) of given example."""
  return CLASSES[get_label(example)]

def get_encoded_image(example):
  """Get the image data (encoded jpg) of given example."""
  image_path = example[0]
  return tf.gfile.GFile(image_path, 'rb').read()

def get_image(example):
  """Get image as np.array of pixels for given example."""
  return plt.imread(io.BytesIO(get_encoded_image(example)), format='jpg')

def display_images(images_and_classes, cols=5):
  """Display given images and their labels in a grid."""
  rows = int(math.ceil(len(images_and_classes) / cols))
  fig = plt.figure()
  fig.set_size_inches(cols * 3, rows * 3)
  for i, (image, flower_class) in enumerate(images_and_classes):
    plt.subplot(rows, cols, i + 1)
    plt.axis('off')
    plt.imshow(image)
    plt.title(flower_class)

NUM_IMAGES = 15 #@param {type: 'integer'}
display_images([(get_image(example), get_class(example))
               for example in TRAIN_EXAMPLES[:NUM_IMAGES]])

## Crie o modelo

Vamos carregar um módulo de vetor de características de imagens do [TF Hub](https://tensorflow.org/hub), empilhar um classificador linear nele e adicionar operações de treinamento e avaliação. A célula abaixo constrói um grafo do TF descrevendo o modelo e seu treinamento, mas não executa o treinamento (isso será feito na próxima etapa).

In [ ]:
LEARNING_RATE = 0.01

tf.reset_default_graph()

# Load a pre-trained TF-Hub module for extracting features from images. We've
# chosen this particular module for speed, but many other choices are available.
image_module = hub.Module('https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2')

# Preprocessing images into tensors with size expected by the image module.
encoded_images = tf.placeholder(tf.string, shape=[None])
image_size = hub.get_expected_image_size(image_module)


def decode_and_resize_image(encoded):
  decoded = tf.image.decode_jpeg(encoded, channels=3)
  decoded = tf.image.convert_image_dtype(decoded, tf.float32)
  return tf.image.resize_images(decoded, image_size)


batch_images = tf.map_fn(decode_and_resize_image, encoded_images, dtype=tf.float32)

# The image module can be applied as a function to extract feature vectors for a
# batch of images.
features = image_module(batch_images)


def create_model(features):
  """Build a model for classification from extracted features."""
  # Currently, the model is just a single linear layer. You can try to add
  # another layer, but be careful... two linear layers (when activation=None)
  # are equivalent to a single linear layer. You can create a nonlinear layer
  # like this:
  # layer = tf.layers.dense(inputs=..., units=..., activation=tf.nn.relu)
  layer = tf.layers.dense(inputs=features, units=NUM_CLASSES, activation=None)
  return layer


# For each class (kind of flower), the model outputs some real number as a score
# how much the input resembles this class. This vector of numbers is often
# called the "logits".
logits = create_model(features)
labels = tf.placeholder(tf.float32, [None, NUM_CLASSES])

# Mathematically, a good way to measure how much the predicted probabilities
# diverge from the truth is the "cross-entropy" between the two probability
# distributions. For numerical stability, this is best done directly from the
# logits, not the probabilities extracted from them.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
cross_entropy_mean = tf.reduce_mean(cross_entropy)

# Let's add an optimizer so we can train the network.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
train_op = optimizer.minimize(loss=cross_entropy_mean)

# The "softmax" function transforms the logits vector into a vector of
# probabilities: non-negative numbers that sum up to one, and the i-th number
# says how likely the input comes from class i.
probabilities = tf.nn.softmax(logits)

# We choose the highest one as the predicted class.
prediction = tf.argmax(probabilities, 1)
correct_prediction = tf.equal(prediction, tf.argmax(labels, 1))

# The accuracy will allow us to eval on our test set. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Treine a rede

Agora que o modelo foi criado, vamos treiná-lo e ver o desempenho para o conjunto de teste.

In [ ]:
# How long will we train the network (number of batches).
NUM_TRAIN_STEPS = 100 #@param {type: 'integer'}
# How many training examples we use in each step.
TRAIN_BATCH_SIZE = 10 #@param {type: 'integer'}
# How often to evaluate the model performance.
EVAL_EVERY = 10 #@param {type: 'integer'}

def get_batch(batch_size=None, test=False):
  """Get a random batch of examples."""
  examples = TEST_EXAMPLES if test else TRAIN_EXAMPLES
  batch_examples = random.sample(examples, batch_size) if batch_size else examples
  return batch_examples

def get_images_and_labels(batch_examples):
  images = [get_encoded_image(e) for e in batch_examples]
  one_hot_labels = [get_label_one_hot(e) for e in batch_examples]
  return images, one_hot_labels

def get_label_one_hot(example):
  """Get the one hot encoding vector for the example."""
  one_hot_vector = np.zeros(NUM_CLASSES)
  np.put(one_hot_vector, get_label(example), 1)
  return one_hot_vector

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(NUM_TRAIN_STEPS):
    # Get a random batch of training examples.
    train_batch = get_batch(batch_size=TRAIN_BATCH_SIZE)
    batch_images, batch_labels = get_images_and_labels(train_batch)
    # Run the train_op to train the model.
    train_loss, _, train_accuracy = sess.run(
        [cross_entropy_mean, train_op, accuracy],
        feed_dict={encoded_images: batch_images, labels: batch_labels})
    is_final_step = (i == (NUM_TRAIN_STEPS - 1))
    if i % EVAL_EVERY == 0 or is_final_step:
      # Get a batch of test examples.
      test_batch = get_batch(batch_size=None, test=True)
      batch_images, batch_labels = get_images_and_labels(test_batch)
      # Evaluate how well our model performs on the test set.
      test_loss, test_accuracy, test_prediction, correct_predicate = sess.run(
        [cross_entropy_mean, accuracy, prediction, correct_prediction],
        feed_dict={encoded_images: batch_images, labels: batch_labels})
      print('Test accuracy at step %s: %.2f%%' % (i, (test_accuracy * 100)))

In [ ]:
def show_confusion_matrix(test_labels, predictions):
  """Compute confusion matrix and normalize."""
  confusion = sk_metrics.confusion_matrix(
    np.argmax(test_labels, axis=1), predictions)
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = list(CLASSES.values())
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

show_confusion_matrix(batch_labels, test_prediction)

## Previsões incorretas

Vamos ver melhor os exemplos de teste que o modelo errou.

- Há algum exemplo rotulado incorretamente no conjunto de teste?
- Há algum dado incorreto no conjunto de teste – imagens que não são realmente de flores?
- Há imagens em que você consegue entender por que o modelo cometeu um erro?

In [ ]:
incorrect = [
    (example, CLASSES[prediction])
    for example, prediction, is_correct in zip(test_batch, test_prediction, correct_predicate)
    if not is_correct
]
display_images(
  [(get_image(example), "prediction: {0}\nlabel:{1}".format(incorrect_prediction, get_class(example)))
   for (example, incorrect_prediction) in incorrect[:20]])

## Exercícios – Aprimore o modelo

Treinamos um modelo de referência. Agora, vamos tentar aprimorá-lo para alcançar uma exatidão melhor (lembre-se de que você precisará executar as células novamente quando fizer uma alteração).

### Exercício 1 – Tente usar um modelo de imagens diferente.

Com o TF Hub, é simples testar alguns modelos de imagens diferentes. Basta substituir o identificador `"https://tfhub.dev/google/imagenet/mobilenet_v2_050_128/feature_vector/2"` na chamada a  `hub.Module()` pelo identificador de um módulo diferente e executar novamente todo o código. Confira todos os módulos de imagens disponíveis em [tfhub.dev](https://tfhub.dev/s?module-type=image-feature-vector).

Uma boa opção pode ser um dos outros [módulos MobileNet V2](https://tfhub.dev/s?module-type=image-feature-vector&network-architecture=mobilenet-v2). Diversos módulos, incluindo os módulos MobileNet, foram treinados com o [dataset ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012), que contém mais de 1 milhão de imagens e mil classes. Ao escolher uma arquitetura de rede, há uma contrapartida entre velocidade e exatidão da classificação: modelos como MobileNet ou NASNet Mobile são pequenos e rápidos, enquanto arquiteturas mais tradicionais, como Inception e ResNet, foram criadas para proporcionar uma alta exatidão.

Para a arquitetura maior Inception V3, você também pode explorar as vantagens de pré-treinar com um domínio mais próximo da sua própria tarefa – e ela também está disponível como um [módulo treinado com o dataset iNaturalist](https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1) de plantas e animais.

### Exercício 2 – Acrescente uma camada oculta.

Empilhe uma camada oculta entre as características de imagem extraídas e o classificador linear (na função `create_model()` acima). Para criar uma camada oculta não linear com, por exemplo, 100 nós, use [tf.layers.dense](https://www.tensorflow.org/api_docs/python/tf/compat/v1/layers/dense), com units definido como 100 e activation definido como `tf.nn.relu`. Alterar o tamanho da camada oculta afeta a exatidão do teste? Adicionar uma segunda camada oculta aumenta a exatidão?

### Exercício 3 – Altere os hiperparâmetros.

Aumentar *o número de passos de treinamento*  aumenta a exatidão final? Você pode *alterar a taxa de aprendizado* para fazer seu modelo convergir mais rapidamente? O *tamanho do lote* do treinamento afeta o desempenho do modelo?

### Exercício 4 – Tente usar um otimizador diferente.

Substitua o GradientDescentOptimizer básico por um otimizador mais sofisticado, como [AdagradOptimizer](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdagradOptimizer). Isso faz alguma diferença no treinamento do modelo? Se você quiser saber mais sobre as vantagens de diferentes algoritmos de otimização, confira [esta postagem](http://ruder.io/optimizing-gradient-descent/).

## Quer aprender mais?

Se você tiver interesse em uma versão mais avançada deste tutorial, confira o [tutorial de retreinamento de imagens do TensorFlow](https://www.tensorflow.org/hub/tutorials/image_retraining), que ensina como visualizar o treinamento usando o TensorBoard, técnicas avançadas, como ampliação do dataset por meio da distorção de imagens, e substituição do dataset Flowers para um classificador de imagens aprender com seu próprio dataset.

Saiba mais sobre o TensorFlow em [tensorflow.org](http://tensorflow.org) e confira a documentação da API do TF Hub, disponível em [tensorflow.org/hub](https://www.tensorflow.org/hub/). Veja os módulos do TensorFlow Hub disponíveis em [tfhub.dev](http://tfhub.dev), incluindo outros módulos de vetor de características de imagens e módulos de embeddings de texto.

Confira também o curso [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/), que é uma introdução prática e acelerada ao aprendizado de máquina do Google.